# Fusion Optimisée (Chunking + Dédoublonnage)
Ce script gère les gros volumes de données en :
1. Chargeant et dédoublonnant le fichier DPE (ne garde que le DPE le plus récent par adresse).
2. Lisant le fichier Conso par blocs (chunks) pour éviter de saturer la mémoire.
3. Ecrivant le résultat au fur et à mesure.

In [ ]:
import pandas as pd
import os
import sys
import subprocess

# Chemins des fichiers
path_conso = r'c:\Users\elean\OneDrive\Bureau\DPE\conso_geocoded_brut.csv'
path_dpe = r'c:\Users\elean\OneDrive\Bureau\DPE\dpe03existant.csv'
path_output = r'c:\Users\elean\OneDrive\Bureau\DPE\fusion_dpe_conso.csv'

In [ ]:
def process_dpe(path_dpe):
    print("Chargement et préparation du fichier DPE...")
    # On sélectionne les colonnes utiles si possible pour gagner de la mémoire
    # Ici on charge tout mais on pourrait optimiser usecols
    try:
        df_dpe = pd.read_csv(path_dpe, sep=',', low_memory=False)
        
        original_len = len(df_dpe)
        print(f"DPE chargé : {original_len} lignes")
        
        # Vérification de la colonne identifiant
        if 'identifiant_ban' not in df_dpe.columns:
            raise ValueError("Colonne 'identifiant_ban' manquante dans le fichier DPE")
            
        # Dédoublonnage : Garder le DPE le plus récent pour chaque identifiant_ban
        # On suppose qu'il y a une colonne de date pour trier, sinon on garde le premier/dernier arbitrairement
        if 'date_etablissement_dpe' in df_dpe.columns:
            print("Dédoublonnage basé sur la date d'établissement...")
            df_dpe['date_etablissement_dpe'] = pd.to_datetime(df_dpe['date_etablissement_dpe'], errors='coerce')
            df_dpe = df_dpe.sort_values('date_etablissement_dpe', ascending=False)
            
        df_dpe = df_dpe.drop_duplicates(subset=['identifiant_ban'], keep='first')
        print(f"Après dédoublonnage : {len(df_dpe)} lignes (Doublons supprimés : {original_len - len(df_dpe)})")
        
        return df_dpe
    except Exception as e:
        print(f"Erreur traitement DPE: {e}")
        return None

# Exécution du traitement DPE
df_dpe_clean = process_dpe(path_dpe)

In [ ]:
if df_dpe_clean is not None:
    print("Début du traitement par chunks du fichier Conso...")
    
    chunksize = 100000  # Taille des blocs à traiter
    first_chunk = True
    total_processed = 0
    total_matches = 0
    
    try:
        # Lecture du fichier Conso par blocs
        with pd.read_csv(path_conso, sep=',', chunksize=chunksize, low_memory=False) as reader:
            for i, chunk in enumerate(reader):
                # Fusion avec le référentiel DPE propre
                merged_chunk = pd.merge(
                    chunk, 
                    df_dpe_clean, 
                    left_on='result_id', 
                    right_on='identifiant_ban', 
                    how='inner'
                )
                
                matches = len(merged_chunk)
                total_matches += matches
                total_processed += len(chunk)
                
                # Ecriture dans le fichier de sortie
                if matches > 0:
                    mode = 'w' if first_chunk else 'a'
                    header = first_chunk
                    merged_chunk.to_csv(path_output, mode=mode, index=False, header=header)
                    first_chunk = False
                
                print(f"Chunk {i+1} traité : {len(chunk)} lignes lues -> {matches} correspondances trouvées.")
                
        print(f"\nTraitement terminé !")
        print(f"Total lignes lues Conso : {total_processed}")
        print(f"Total correspondances sauvegardées : {total_matches}")
        print(f"Fichier disponible ici : {path_output}")
        
    except Exception as e:
        print(f"Erreur lors du traitement par chunks : {e}")
else:
    print("Impossible de procéder : le traitement DPE a échoué.")

In [ ]:
import pandas as pd
print("\n--- Visualisation du fichier fusionné (Extrait) ---")
try:
    if 'path_output' not in locals():
        path_output = r'c:\Users\elean\OneDrive\Bureau\DPE\fusion_dpe_conso.csv'
    
    # On charge uniquement les 20 premières lignes pour visualiser sans tout charger
    df_preview = pd.read_csv(path_output, nrows=20, low_memory=False)
    print(f"Aperçu des 5 premières lignes :")
    print(df_preview.head())
    
    print(f"\nInfos sur les colonnes :")
    print(df_preview.info())
except Exception as e:
    print(f"Erreur à l'affichage : {e}")